<a href="https://colab.research.google.com/github/aerosense-ai/notebooks/blob/main/post_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aerosense-ai/notebooks/blob/main/pre_process.ipynb)

In [ ]:
!python --version

<IPython.core.display.Javascript object>

Python 3.8.10


In [ ]:
# Avoids scroll-in-the-scroll in the entire Notebook
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

# Dependences and new code

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install --upgrade git+https://github.com/aerosense-ai/aerosense-tools.git@feature/add-combined-sensor-sessions

In [ ]:
import datetime as dt

from aerosense_tools.queries import BigQuery
from aerosense_tools import plots
from aerosense_tools.preprocess import RawSignal
from aerosense_tools.preprocess import SensorMeasurementSession

client = BigQuery()

In [ ]:
sensor_types_metadata = {
    "accelerometer": {
        "description": "Accelerometer Measurement Series",
        "variable": "Acceleration [m/s²]",
        "sensors": ["Acc. Direction 1", "Acc. Direction 2", "Acc. Direction 3"],
        },
    "barometer": {
        "description": "Barometer Measurement Series",
        "variable": "Pressure [Pa]",
        "sensors": ["Abs. Baro. Sensor {}".format(i) for i in range(40)],
    },
    "barometer_thermometer": {
        "description": "Temperature Measurement Series from Barometer Sensors",
        "variable": "Temperature [C]",
        "sensors": ["Temp. Sensor {}".format(i) for i in range(40)]
    },
    "connection_statistics": {
        "description": "Connection Statistics",
        "variable": [
            "Transmitting Power [W]",
            "Allocated Heap Memory [Bytes]",
            "Received signal strength indication: Raw [dBm]",
            "Received signal strength indication: Filtered [dBm]"
        ],
        "sensors": ["Conn. Stat. {}".format(i) for i in range(4)]
    },
    "battery_info": {
        "description": "Battery State information",
        "variable": [
            "Voltage [V]",
            "Number of cylces",
            "Battery Capacity [%]"
        ],
        "sensors": ["Batt. info. {}".format(i) for i in range(3)]
    },
    "differential_barometer": {
        "description": "Differential Pressure Measurement Series",
        "variable": "Differential Pressure [Pa]",
        "sensors": ["Diff. Baro. Sensor {}".format(i) for i in range(5)],
        },
    "gyroscope": {
        "description": "Gyroscope Measurement Series",
        "variable": "Angular Velocity [s⁻¹]",
        "sensors": ["Gyro. Direction 1", "Gyro. Direction 2", "Gyro. Direction 3"],
        },
    "magnetometer": {
        "description": "Magnetometer Measurement Series",
        "variable": "Magnetic Momemnt [A⋅m²]",
        "sensors": ["Mag. Direction 1", "Mag. Direction 2", "Mag. Direction 3"],
        },
}

# Data Playground
Lets play with Raw data

In [ ]:
print("Available sensor types are: {}".format(client.get_sensor_types()))
print("Available installations are: {}".format(client.get_installations()))

Available sensor types are: ['accelerometer', 'barometer', 'barometer_thermometer', 'battery_info', 'connection_statistics', 'differential_barometer', 'gyroscope', 'magnetometer', 'microphone']
Available installations are: [{'label': 'aventa-turbine-test (Turbine 0)', 'value': 'aventa-turbine-test'}, {'label': 'my-next-test-installation (Turbine 0)', 'value': 'my-next-test-installation'}, {'label': 'ost-wt-evaluation (Turbine 0)', 'value': 'ost-wt-evaluation'}, {'label': 'ost-wt-tests (Turbine 0)', 'value': 'ost-wt-tests'}, {'label': 'pbl-test (Turbine unknown)', 'value': 'pbl-test'}, {'label': 'predep-test-20221130 (Turbine unknown)', 'value': 'predep-test-20221130'}, {'label': 'test-installation (Turbine 0)', 'value': 'test-installation'}]


In [ ]:
measuremnt_sessions={}
raw_signals={}
measuremnt_session_times={}

In [ ]:
sensor_type="differential_barometer"

Get data from Big Query:

In [ ]:
node="1"
installation = "aventa-turbine-test"
measurement_day = dt.datetime(2023, 2, 24)
query_period = dt.timedelta(days=2)

df, data_limit_applied = client.get_sensor_data(
    installation_reference=installation,
    node_id=node,
    sensor_type_reference=sensor_type,
    start=measurement_day,
    finish=measurement_day+query_period,
    row_limit=10000000,
)

In [ ]:
data_columns =  df.columns[df.columns.str.startswith('f')].tolist()
signal_df = df[["datetime"]+data_columns].set_index('datetime')
signal_df.columns = sensor_types_metadata[sensor_type]["sensors"]
raw_signal = RawSignal(signal_df, sensor_type)
raw_signal.measurement_to_variable()

raw_signals[sensor_type]=raw_signal
measuremnt_sessions[sensor_type], measuremnt_session_times[sensor_type] = raw_signal.extract_measurement_sessions()

420061 2023-02-24 07:53:56.844722


In [ ]:
measuremnt_session_times[sensor_type]

Plot a session. 

*Note: session might be too long (too much data to plot) and colab will kill the plotting*

In [ ]:
session=18
fig=measuremnt_sessions[sensor_type][session].plot(
    sensor_types_metadata,
#   plot_max_time=dt.timedelta(minutes=1),
#   plot_start_offset=dt.timedelta(minutes=8),
    sensor_names='Diff. Baro. Sensor 2'
    )
fig.update_layout(height=800)
fig.show()

In [ ]:
max_diff=[measuremnt_sessions["differential_barometer"][session].sensor_statistics.loc['max'].max() for session in measuremnt_session_times["differential_barometer"].index]
operations_df = measuremnt_session_times["differential_barometer"].copy()
operations_df["Max Diff. pressure"] = max_diff
operations_df

Check opertaional conditions of the WT

In [ ]:
min_accelerations=[measuremnt_sessions["accelerometer"][session].sensor_statistics["Acc. Direction 2"]["min"]/9.80665 for session in measuremnt_session_times["accelerometer"].index]
operations_df = measuremnt_session_times["accelerometer"].copy()
operations_df["Min Acc. Dir. 2 [g]"] = min_accelerations
operations_df


In [ ]:
barometer_processed = measuremnt_sessions["barometer"][206].dataframe
barometer_processed.columns=range(40)

In [ ]:
barometer_processed -= 96.7e3

In [ ]:
import pandas as pd
import plotly.express as px

barometer_coordinates = pd.DataFrame(
{
    "x":[-0.176219681501034,-0.162835974532066,-0.148699716661985,-0.135229588214901,-0.12119040735309,-0.107587403498112,-0.0933713181476206,-0.080137967474052,0.0523686022871351,0.0659076345682686,0.0794134788054948,0.0891034825248706,0.088950636331581,0.080268459869684,0.0684441178816674,0.0557708451669033,-0.0622999945773802,-0.0487315579575698,-0.0346432659490448,-0.0209254001331136,-0.00672858017667943,0.00676519205633344,0.0211079272200362,0.034717506840704,0.0388002996281375,0.0251135878479515,0.0113468398976768,-0.00261364170933859,-0.0166256579081756,-0.0305996414920209,-0.0446779426168727,-0.0586219179151267,-0.0756052562810064,-0.0894800576710868,-0.103413075068334,-0.117194886194055,-0.131145319207758,-0.144787504527316,-0.158644885888325,-0.172294381163741],
    "y":[0.0128786481588428,0.0140160958114696,0.0153022562393278,0.0165674546228921,0.0183806678486176,0.0192921983013588,0.0200244660166134,0.0211083282425242,0.0243512654271175,0.0217492838901699,0.0160145528599462,0.00544179547389913,-0.00947928544444861,-0.0208612457217944,-0.0291223328106538,-0.0350889304966803,0.0230140404564016,0.0239561289710152,0.0249459566541184,0.0255654575171109,0.02613364352823,0.0265329731115964,0.0266153092325157,0.0261554208905004,-0.0410860757132368,-0.0440193542023281,-0.0464697126467811,-0.0477951850129886,-0.0491130947974371,-0.0490433169408885,-0.048995286216894,-0.0482195612877382,-0.0473287347673169,-0.0456927189337512,-0.0443723626972239,-0.0421078452958516,-0.0404529178116761,-0.0376291298510196,-0.0353387566300693,-0.0320994476402506],
    "sensor": barometer_processed.columns
 })


fig = px.scatter(barometer_coordinates, x="x", y="y", text="sensor")
fig.update_traces(textposition='top center')
fig.update_layout({"title":"Sensor Locations"})
fig.show()

In [ ]:
barometer_coordinates["x"] *=-1 
barometer_coordinates["y"] *=-1 
barometer_coordinates["x"] +=0.1 

fig = px.scatter(barometer_coordinates, x="x", y="y", text="sensor")
fig.update_traces(textposition='top center')
fig.update_layout({"title":"Sensor Locations"})
fig.show()

In [ ]:
x_pressure_side = barometer_coordinates[barometer_coordinates["y"]<0]["x"].sort_values()
x_suction_side = barometer_coordinates[barometer_coordinates["y"]>0]["x"].sort_values()

suction_side_barometers = barometer_processed.columns[x_suction_side.index]
pressure_side_barometers = barometer_processed.columns[x_pressure_side.index]

In [ ]:
suction_df = barometer_processed[suction_side_barometers]
pressure_df = barometer_processed[pressure_side_barometers]

suction_df = suction_df.drop([12, 24], axis=1)
pressure_df = pressure_df.drop([11], axis=1)

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "colab"

ani_samples = 500

plot_frames = [go.Frame(
    data=[
        go.Scatter(            
            x=x_pressure_side,
            y=pressure_df.iloc[i],
            text=pressure_df.columns,
            textposition='top center',
            name="Pressure side ",
            mode='lines+markers+text',
            marker=dict(color="red")
        ),
        go.Scatter(            
            x=x_suction_side,
            y=suction_df.iloc[i],
            text=suction_df.columns,
            textposition='top center',
            name="Suction side ",
            mode='lines+markers+text',
            marker=dict(color="blue")
        )        
    ],
    name=str(suction_df.index[i])
    )
for i in range(ani_samples)]


sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Time:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 60},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": [{"args": [
        [str(suction_df.index[i])],
        {"frame": {"duration": 60, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 0}}
    ],
        "label": str(suction_df.index[i]),
        "method": "animate"} for i in range(ani_samples)]
}
menus_dict={
    "type":"buttons",
    "buttons":[
        {
            "label":"\u25B6",
            "method":"animate",
            "args":[
                None,
                {
                    "frame": {"duration": 60, "redraw": True},
                    "transition": {"duration": 0}
                 }
            ]
        },
        {
            "label":"Pause",
            "method":"animate",
            "args":[
                [None],
                {
                    "frame": {"duration": 0, "redraw": True},
                    "transition": {"duration": 0},
                    "mode": "immediate",
                    "transition": {"duration": 0}
                }
            ]
        }
    ],
    "yanchor":"bottom",
    "y":-0.5,   
}

layout_dict={
    "title_text":"Cp Curve",
    "hovermode":"closest",
    "xaxis":{
        "title":"Chordwise sensor postition [m/1m]",
        "range":[0, 0.35],
        "autorange":False
    },
    "yaxis":{
        "title":"Cp Curve",
        "range":[2.5e3, -1e3],
        "autorange":False
    },
    "sliders":[sliders_dict],
    "updatemenus":[menus_dict]
}

fig = go.Figure(
  data = plot_frames[0]["data"],
  layout = layout_dict,
  frames = plot_frames
)

fig.show()

In [8]:
from google.cloud import storage
import json

client = storage.Client()
bucket = client.get_bucket('panel-codes-data')
blob = bucket.blob('results/polars_aerosense_aventa_node1.json')
content = blob.download_as_string()
aventa_polars = json.loads(content)

In [36]:
simulation_number=30
aoa=aventa_polars['output_values']['aoa'][simulation_number]
sim_result=aventa_polars['output_values']['cp'][simulation_number]
sim_df=pd.DataFrame(
    {
        "Cp":sim_result['cp'],
        "x-coord":sim_result['x-coord'],
        "y-coord":sim_result['y-coord']
    }
)


In [38]:
import plotly.express as px
pressure_df=sim_df[sim_df['y-coord']<=0]
suction_df=sim_df[sim_df['y-coord']>=0]

layout_dict={
    "title_text":"Cp Curve for AoA of {}".format(aoa),
    "xaxis":{"title":"Chordwise sensor postition [m/1m]"},
    "yaxis":{
        "title":"Cp",
        "range":[2, -2],
        "autorange":False
    }
}


fig=px.scatter(height=800)
fig.add_scatter(x=pressure_df["x-coord"], y=pressure_df["Cp"], marker={"color":"red"}, mode="lines", name="Pressure Side")
fig.add_scatter(x=suction_df["x-coord"], y=suction_df["Cp"], marker={"color":"blue"}, mode="lines", name="Suction Side")
fig.update_layout(layout_dict)
fig.show()
